In [15]:
import re
import string
import data_utils

In [27]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['real_estate_data']
read_collection = db['real_estate_query_data']
write_collection = db['real_estate_query_data_normailze']

In [24]:
def normalize(text, text_type="none"):
    text = data_utils.compound2unicode(text)
    text = re.sub('Mười','10', text)
    text = re.sub('mười','10', text)
    text = data_utils.remove_vietnamese_accent(text)
#     print('haha', text)
    if text_type=="price":
        low, high = normalize_price(text)
        return low, high
    
    elif text_type=="area":
        text = normalize_area(text)
        
    elif text_type=="ward":
        text = normalize_ward(text)  
    
    elif text_type=="interior_room"
        text = normalize_room(text)
        
    return text

In [82]:
main_divider = '-'
dividers = ['toi', 'va', '~', 'hoac']
currency_unit = ['ti', 'ty', 'trieu', 'tr', 'nghin', 'ngan', 'k']
maping_num = {'mot':'1', 'hai':'2', 'ba':'3', 'bon':'4', 'nam':'5', 'sau':'6', 'bay':'7', 'tam':'8', 'chin':'9'}
def normalize_price(text):
    for key, value in maping_num.items():
         text = re.sub(r"\b{}\b".format(key), "{}".format(value), text)
    text = text.replace('ti', 'ty')    
    text = text.replace('tram', '#00')
    text = text.replace('trieu', 'tr')
    text = text.replace('muoi', '#0') #Mươi :)))
    text = text.replace(' #', '')
    text = text.replace('ruoi', '5') # rưỡi
    text = text.replace('mot', '1') #Mốt nhé
    text = text.replace('k', 'kk')
    text = text.replace('nghin', 'kk')
    text = text.replace('ngan', 'kk')
    text = text.replace(',', '.')
    
    for div in dividers:
        text = re.sub(div, main_divider, text)
    price_list = list()
    arr = text.split(main_divider)
#     print('Divided: ', arr)
    biggest_unit = None
    for element in reversed(arr):
        prices = powerful_split_price(element)
        print('Splited: ', prices)
        for price in reversed(prices):
            value, unit = normalize_price_unit(price, biggest_unit)
            price_list.append(value)
            biggest_unit = unit
    
    low, high = min(price_list), max(price_list)
    if low == high: high = None
    return low, high

re_num = '\d+(\s\.\s\d+)?'
re_hud = re.compile('(\d+(\s\.\s\d+)?\skk)')
re_mil = re.compile('(\d+(\s\.\s\d+)?\str)')
re_bil = re.compile('(\d+(\s\.\s\d+)?\sty)')
def powerful_split_price(text):
    text = text.strip()
    idx_bil = [0] + [i.start() for i in re.finditer(re_bil, text)] + [len(text)]
    idx_mil = [0] + [i.start() for i in re.finditer(re_mil, text)] + [len(text)]
    idx_hud = [0] + [i.start() for i in re.finditer(re_hud, text)] + [len(text)]
#     print('idx_bil', idx_bil)
#     print('idx_mil', idx_mil)
#     print('idx_hud', idx_hud)
    price_list = list()
    if len(idx_bil) > 2:
        for i in range(0, len(idx_bil) - 1):
            price = text[idx_bil[i]:idx_bil[i+1]]
            if price != '':
                price_list.append(price)
    elif len(idx_mil) > 2:
        for i in range(0, len(idx_mil) - 1):
            price = text[idx_mil[i]:idx_mil[i+1]]
            if price != '':
                price_list.append(price)
    elif len(idx_hud) > 2:
        for i in range(0, len(idx_hud) - 1):
            price = text[idx_hud[i]:idx_hud[i+1]]
            if price != '':
                price_list.append(price)
    elif text != '':
        price_list.append(text)      
    return price_list
    
maping_unit = {'ty': 1000000000, 'tr': 1000000, 'kk':1000}
def normalize_price_unit(text, pre_unit='kk'):
    if text == '':
        return 0, None
    final_value = 0
    arr = text.split(' ')
    current_unit = pre_unit
    num_list = [float(re.sub(' ','', i.group())) for i in re.finditer('\d+(\s.\s\d+)?', text)]
    unit_list = [i.group() for i in re.finditer('[a-z]+', text)]
    print('Num list: ', num_list)
    print('Unit list: ', unit_list)
    if len(unit_list) == 0:
        final_value = num_list[-1] * maping_unit[pre_unit]
        return final_value, pre_unit
    
    odd_unit = 'kk'
    for i in range(min(len(num_list), len(unit_list))):
        num = num_list[i]
        unit = unit_list[i]
        if unit in maping_unit.keys():
            final_value += maping_unit[unit]*num
            odd_unit = unit
    if len(num_list) > len(unit_list):
        odd = num_list[len(unit_list)]
        if odd < 10:
            final_value += maping_unit[odd_unit]*odd/10
        else:
            final_value += maping_unit[odd_unit]*odd/1000               
#     print(final_value, odd_unit)
    return final_value, odd_unit

In [83]:
area_divider = ['dai','rong','nhan','\*']

re_num_with_unit = '(\d+(\s\.\s\d+)?)(\s(m\^|km\^|m|km|ha))?'
re_num_n_x = '(' + re_num_with_unit + '\sx\s)'
re_powerful_x =  re_num_n_x + '+' + re_num_with_unit

def normalize_area(text):
    for div in area_divider:
        text = text.replace(div,'x')
    for key, value in maping_num.items():
         text = re.sub(r"\b{}\b".format(key), "{}".format(value), text)
    text = text.replace(',','.')
    text = text.replace('x',' x ')
    text = text.replace('  ',' ')
    text = text.replace('kilo met','km')
    text = text.replace('met','m')   
    text = text.replace('vuong','2')
    text = text.replace('m 2','m^')
    text = text.replace('m2','m^')
    text = text.replace('hecta','ha')
    text = text.replace('hec ta','ha')
    text = text.replace('hec','ha')
    area_list = list()
    pre_RHS = '0' #right hand side

    if 'x' in text:
        x_group = [i.group() for i in re.finditer(re_powerful_x, text)]
        for x in x_group:
            text = text.replace(x,'')
            area_list += normalize_area_x(x)
    
    non_x_group = [i.group() for i in re.finditer(re_num_with_unit, text)]
    for x in non_x_group:  
        area_list.append(normalize_area_non_x(x))
            
    return area_list

def normalize_area_x(text):
    factor = 1
    if 'km' in text:
        factor = 1000000
    text = re.sub('[^\d\.x]','', text)
    area_list = list()
    arr = [float(i) for i in text.split('x')]
    for i in range(len(arr) -1):
        area = arr[i]*arr[i+1]*factor
        area_list.append(area)
    return area_list

def normalize_area_non_x(text):
    factor = 1
    if 'km' in text:
        factor = 1000000
    elif 'ha' in text:
        factor = 10000
    text = re.sub('[^\d\.]','', text)
    return float(text)*factor

In [84]:
re_rooms = [
    r"s(an)?\W*t(huong)?",
    r"san\b",
    r"(p(hong)?)?\W*t(ro)?|\btro\b",
    r"(p(hong)?)?\W*n(gu)?|\bngu\b",
    r"(p(hong)?)?\W*g(ia[tc])?|\bgia[tc]\b",
    r"(p(hong)?)?\W*t(ho)?\b|\btho\b",
    r"(p(hong)?)?\W*k(hach)?|\bkhach\b",
    r"n(ha)?\W*k(ho)?|\bkho\b",
    r"gara|o\W*to|xe\W*hoi",
    r"xe(\W*may)?",
    r"ki\W*o[ts]",
    r"(gieng\W*)?troi",
    r"van\W*phong|k(inh)?\W+d(oanh)?",
    r"ba[nl]g?\W*co(ng?|l)",
    r"(p(hong)?)?\W*(b(ep)?|\ban\b)|\bbep\W*an\b",
    r"(p(hong)?)?\W*(tam|v(e)?\W*s(inh)?|wc|toi?ll?e?t)",
    r"(p(hong)?)?\W+l(am)?\W+v(iec)?",
    r"(p(hong)?)?\W+s(inh)?\W+h(oat)?"]

room_name = [
    "san thuong",
    "san",
    "phong tro",
    "phong ngu",
    "phong giat",
    "phong tho",
    "phong khach",
    "nha kho",
    "gara",
    "xe may",
    "kiots",
    "gieng troi",
    "van phong",
    "ban cong",
    "bep an",
    "nha ve sinh",
    "phong lam viec",
    "phong sinh hoat"
]

#Nhớ xoá ban công

def normalize_room(text):
    text = text.split()
    temp = text[-1]
    text = ' '.join(text[:-1])

    for key, value in maping_num.items():
        text = re.sub(r"\b{}\b".format(key), "{}".format(value), text)
        
    text = text + ' ' + temp 
    num_arr = [i.group() for i in re.finditer('\d+(\s\.\s\d+)?',text)]
    num_arr = [int(float(i.replace(' ',''))) for i in num_arr]
    if len(num_arr) > 0:
        num_room = min(num_arr)
    else:
        num_room = 1
    current_key = ''
    current_idx = -1
    for idx, regex in enumerate(re_rooms):
        print(text)
        keys = [i.group() for i in re.finditer(regex, text)]
        print(idx, keys)
        if len(keys) > 0 and len(current_key) < len(max(keys, key = len)):
            current_key = max(keys, key = len)
            current_idx = idx
    if current_idx == -1:
        return None, num_room
    return room_name[current_idx], num_room
    

In [130]:
def lcs(X, Y):
    X =X.replace(' ','')
    Y =Y.replace(' ','')
    m = len(X)
    n = len(Y)   
    # declaring the array for storing the dp values
    L = [[None]*(n+1) for i in range(m+1)]

    """Following steps build L[m+1][n+1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m+1):
        for j in range(n+1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])

    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1]
    return L[m][n]

In [131]:
re_floor = [
    "tang", "lau", "tam", "me", "cap 4",
    "tang gac",
    "tang tret",
    "tang lung",
    "tang ham",
    "ban cong",
    "san thuong",
]
floor_name = [
    "tang", "tang", "tang", "tang", "tang",
    "gac",
    "tret",
    "lung",
    "ham",
    "ban cong",
    "san thuong",
]

def normalize_floor(text):
    half = 0;
    if 'ruoi' in text:
        half = 0.5
        text = re.sub(r"\b{}\b".format('ruoi'), '', text)
    text = text.split()
    temp = text[-1]
    text = ' '.join(text[:-1])

    for key, value in maping_num.items():
        text = re.sub(r"\b{}\b".format(key), "{}".format(value), text)      
    text = text + ' ' + temp 
    num_arr = [i.group() for i in re.finditer('\d+(\s\.\s\d+)?',text)]
    num_arr = [int(float(i.replace(' ',''))) for i in num_arr]
    if len(num_arr) > 0:
        num_floor = min(num_arr)
    else:
        num_floor= 1
    lcs_list = list()
    for regex in re_floor:
        lcs_list.append(lcs(regex, text))
    print(text)
    print(lcs_list)
    best_match = max(lcs_list)
    if best_match == 0:
        return None, num_floor
    else:
        return floor_name[lcs_list.index(best_match)], num_floor + half

In [132]:
normalize_floor('2tang')

 2tang
[4, 1, 2, 0, 1, 4, 4, 4, 4, 3, 3]


('tang', 2)

In [50]:
count = 0
for document in read_collection.find():
#     document["price"] = normalize_price(document["price"])
    print(document["usable_area"])
#     write_collection.insert_one(document)
    count = count+1
    if count > 100:
        break

: 109 m
: 73,3 m
: 112 m
: 69 m
: 76 m
: 45 m
: 71 m
: 93 m
: 55 m
: 30 m
: 114 m
: 61 m
: 75 m
: 40 m
: 130 m
: 44 m
: 65 m
: 95 m
: 115 m
: 64 m
: 46 m
: 75 m
: 110 m
: 50 m
: 65 m
: 77 m
: 69 m
: 58 m
: 35 m
: 60 m
: 30 m
: 69 m
: 48 m
: 74 m
: 74 m
: 74 m
: 56 m
: 45 m
: 65 m
: 54 m
: 70 m
: 48 m
: 65 m
: 65,56 m
: 78,81 m
: 60 m
: 37,26 m
: 80 m
: 65 m
: 35,53 m
: 65,83 m
: 65 m
: 45 m
: 80 m
: 63 m
: 82 m
: 80 m
: 80 m
: 80 m
: 48 m
: 83 m
: 62 m
: 95 m
: 82 m
: 95 m
: 54 m
: 108 m
: 45 m
: 70 m
: 44 m
: 50 m
: 70 m
: 54 m
: 65 m
: 70 m
: 56 m
: 75 m
: 70 m
: 38 m
: 38 m
: 96 m
: 38 m
: 88 m
: 35,53 m
: 90 m
: 88 m
: 88 m
: 72,05 m
: 68,13 m
: 67,9 m
: 72 m
: 97,57 m
: 88 m
: 98 m
: 68,11 m
: 97,55 m
: 68,44 m
: 150 m
: 97,45 m
: 68,66 m
: 68,61 m
